BASICS OF FLASK

In [ ]:
# 1. Create a Flask app that displays "Hello, World!" on the homepage.
# hello.py
from flask import Flask

app = Flask(__name__)

@app.route('/')
def home():
    return "Hello, World!"

if __name__ == "__main__":
    app.run(debug=True)



In [ ]:
# 2. Build a Flask app with static HTML pages and navigate between them.
from flask import Flask, render_template

app = Flask(__name__)

@app.route('/')
def home():
    return render_template("home.html")

@app.route('/about')
def about():
    return render_template("about.html")

if __name__ == "__main__":
    app.run(debug=True)


In [ ]:
# 3. Develop a Flask app that uses URL parameters to display dynamic content.
from flask import Flask

app = Flask(__name__)

@app.route('/user/<name>')
def greet(name):
    return f"Hello, {name.capitalize()}!"

if __name__ == "__main__":
    app.run(debug=True)


In [ ]:
# 4. Create a Flask app with a form that accepts user input and displays it.
from flask import Flask, request, render_template

app = Flask(__name__)

@app.route('/', methods=['GET', 'POST'])
def form():
    if request.method == 'POST':
        name = request.form['name']
        return f"Hello, {name}!"
    return render_template("form.html")

if __name__ == "__main__":
    app.run(debug=True)


In [ ]:
# 5. Implement user sessions in a Flask app to store and display user-specific data
from flask import Flask, session, redirect, url_for, request

app = Flask(__name__)
app.secret_key = 'secret'

@app.route('/')
def index():
    if 'user' in session:
        return f'Logged in as {session["user"]}'
    return 'You are not logged in'

@app.route('/login', methods=['POST', 'GET'])
def login():
    if request.method == 'POST':
        session['user'] = request.form['username']
        return redirect(url_for('index'))
    return '''
        <form method="post">
            <input type=text name=username>
            <input type=submit>
        </form>
    '''

@app.route('/logout')
def logout():
    session.pop('user', None)
    return redirect(url_for('index'))

if __name__ == "__main__":
    app.run(debug=True)


INTERMEDIATE

In [ ]:
# 6. Build a Flask app that allows users to upload files and display them on the website.
from flask import Flask, request, redirect, render_template
import os

app = Flask(__name__)
UPLOAD_FOLDER = 'uploads'
os.makedirs(UPLOAD_FOLDER, exist_ok=True)
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

@app.route('/', methods=['GET', 'POST'])
def upload_file():
    if request.method == 'POST':
        file = request.files['file']
        file.save(os.path.join(app.config['UPLOAD_FOLDER'], file.filename))
        return f"File uploaded: {file.filename}"
    return render_template("upload.html")

if __name__ == '__main__':
    app.run(debug=True)


In [ ]:
# 7. Integrate a SQLite database with Flask to perform CRUD operations on a list of items
from flask import Flask, render_template, request, redirect
import sqlite3

app = Flask(__name__)

def init_db():
    conn = sqlite3.connect('data.db')
    c = conn.cursor()
    c.execute('CREATE TABLE IF NOT EXISTS items (id INTEGER PRIMARY KEY, name TEXT)')
    conn.commit()
    conn.close()
init_db()

@app.route('/')
def index():
    conn = sqlite3.connect('data.db')
    items = conn.execute('SELECT * FROM items').fetchall()
    conn.close()
    return render_template('crud.html', items=items)

@app.route('/add', methods=['POST'])
def add():
    name = request.form['name']
    conn = sqlite3.connect('data.db')
    conn.execute('INSERT INTO items (name) VALUES (?)', (name,))
    conn.commit()
    conn.close()
    return redirect('/')

@app.route('/delete/<int:item_id>')
def delete(item_id):
    conn = sqlite3.connect('data.db')
    conn.execute('DELETE FROM items WHERE id=?', (item_id,))
    conn.commit()
    conn.close()
    return redirect('/')

if __name__ == '__main__':
    app.run(debug=True)


In [ ]:
# 8. Implement user authentication and registration in a Flask app using Flask-Login.
# pip install flask flask-login werkzeug
# auth_app.py
from flask import Flask, render_template, redirect, request, url_for
from flask_login import LoginManager, UserMixin, login_user, login_required, logout_user, current_user
from werkzeug.security import generate_password_hash, check_password_hash

app = Flask(__name__)
app.secret_key = 'secret'

# Flask-Login setup
login_manager = LoginManager()
login_manager.init_app(app)
login_manager.login_view = 'login'

# In-memory user store
users = {}

class User(UserMixin):
    def __init__(self, id, username, password_hash):
        self.id = id
        self.username = username
        self.password_hash = password_hash

@login_manager.user_loader
def load_user(user_id):
    return users.get(user_id)

@app.route('/')
@login_required
def home():
    return f"Hello, {current_user.username}! <a href='/logout'>Logout</a>"

@app.route('/register', methods=['GET', 'POST'])
def register():
    if request.method == 'POST':
        username = request.form['username']
        password = generate_password_hash(request.form['password'])
        user = User(str(len(users)+1), username, password)
        users[user.id] = user
        return redirect(url_for('login'))
    return render_template('register.html')

@app.route('/login', methods=['GET','POST'])
def login():
    if request.method == 'POST':
        username = request.form['username']
        password = request.form['password']
        for user in users.values():
            if user.username == username and check_password_hash(user.password_hash, password):
                login_user(user)
                return redirect(url_for('home'))
        return "Invalid credentials"
    return render_template('login.html')

@app.route('/logout')
@login_required
def logout():
    logout_user()
    return redirect(url_for('login'))

if __name__ == "__main__":
    app.run(debug=True)



In [ ]:
# 9. Create a RESTful API using Flask to perform CRUD operations on resources like books or movies.
pip install flask flask_sqlalchemy
# api_app.py
from flask import Flask, request, jsonify
from flask_sqlalchemy import SQLAlchemy

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///books.db'
db = SQLAlchemy(app)

class Book(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    title = db.Column(db.String(80))
    author = db.Column(db.String(80))

db.create_all()

@app.route('/books', methods=['GET'])
def get_books():
    books = Book.query.all()
    return jsonify([{'id': b.id, 'title': b.title, 'author': b.author} for b in books])

@app.route('/books/<int:id>', methods=['GET'])
def get_book(id):
    book = Book.query.get_or_404(id)
    return jsonify({'id': book.id, 'title': book.title, 'author': book.author})

@app.route('/books', methods=['POST'])
def add_book():
    data = request.get_json()
    new_book = Book(title=data['title'], author=data['author'])
    db.session.add(new_book)
    db.session.commit()
    return jsonify({'message': 'Book added'}), 201

@app.route('/books/<int:id>', methods=['PUT'])
def update_book(id):
    book = Book.query.get_or_404(id)
    data = request.get_json()
    book.title = data['title']
    book.author = data['author']
    db.session.commit()
    return jsonify({'message': 'Book updated'})

@app.route('/books/<int:id>', methods=['DELETE'])
def delete_book(id):
    book = Book.query.get_or_404(id)
    db.session.delete(book)
    db.session.commit()
    return jsonify({'message': 'Book deleted'})

if __name__ == "__main__":
    app.run(debug=True)


In [ ]:
# 10. Design a Flask app with proper error handling for 404 and 500 errors.
# error_app.py
from flask import Flask, render_template

app = Flask(__name__)

@app.errorhandler(404)
def not_found(error):
    return render_template('404.html'), 404

@app.errorhandler(500)
def server_error(error):
    return render_template('500.html'), 500

@app.route('/')
def home():
    return "Home Page"

@app.route('/error')
def trigger_error():
    1 / 0  # Force error



REAL TIME PROJECT

In [ ]:
# 11. Create a real-time chat application using Flask-SocketIO.
pip install flask flask-socketio
# chat_app.py
from flask import Flask, render_template
from flask_socketio import SocketIO, send

app = Flask(__name__)
app.config['SECRET_KEY'] = 'secret'
socketio = SocketIO(app)

@app.route('/')
def index():
    return render_template('chat.html')

@socketio.on('message')
def handle_message(msg):
    print("Message: " + msg)
    send(msg, broadcast=True)

if __name__ == "__main__":
    socketio.run(app, debug=True)



In [ ]:
# 12. Build a Flask app that updates data in real-time using WebSocket connections.
# live_update.py
from flask import Flask, render_template
from flask_socketio import SocketIO
import time, threading

app = Flask(__name__)
socketio = SocketIO(app)

@app.route('/')
def index():
    return render_template('live.html')

def background_task():
    count = 0
    while True:
        socketio.emit('update', {'counter': count})
        count += 1
        time.sleep(2)

@socketio.on('connect')
def connected():
    print("Client connected")

if __name__ == "__main__":
    threading.Thread(target=background_task).start()
    socketio.run(app, debug=True)


In [ ]:
# 13. Implement notifications in a Flask app using websockets to notify users of updates
# notify_app.py
from flask import Flask, render_template
from flask_socketio import SocketIO

app = Flask(__name__)
socketio = SocketIO(app)

@app.route('/')
def index():
    return render_template('notify.html')

@app.route('/send_notification/<msg>')
def send_notification(msg):
    socketio.emit('notification', {'message': msg})
    return "Notification sent!"

if __name__ == "__main__":
    socketio.run(app, debug=True)
